In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pandas
import matplotlib
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.113:7077") \
        .appName("AdityaShirke_A")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

In [7]:
sc = spark_session.sparkContext
lines_sv_en_en = sc.textFile("hdfs://192.168.2.113:9000/europarl/europarl-v7.sv-en.en")
linecount_sv_en_en = lines_sv_en_en.count()
linecount_sv_en_en

1862234

In [8]:
lines_sv_en_sv = sc.textFile("hdfs://192.168.2.113:9000/europarl/europarl-v7.sv-en.sv").cache()
linecount_nl_en_sv = lines_sv_en_sv.count()
linecount_nl_en_sv

1862234

In [9]:
lines_sv_en_en.getNumPartitions()

2

In [10]:
lines_sv_en_sv.getNumPartitions()

3

In [11]:
lines_sv_en_en.take(10)

['Resumption of the session',
 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.',
 "Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.",
 'You have requested a debate on this subject in the course of the next few days, during this part-session.',
 "In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the European Union.",
 "Please rise, then, for this minute' s silence.",
 "(The House rose and observed a minute' s silence)",
 'Madam President, on a point of order.',
 'You will be aware from the press and television that there have be

In [12]:
#A2.1 function to Pre-process the text from both RDDs by doing the following: Lowercase the text and Tokenize the text (split on space)
def lowercase_split(rdd):
    return rdd.map(lambda line: line.lower().split(' '))

preprocessed_text_sv_en_en = lowercase_split(lines_sv_en_en)
print(preprocessed_text_sv_en_en.take(10))
print(preprocessed_text_sv_en_en.count())

preprocessed_text_sv_en_sv = lowercase_split(lines_sv_en_sv)
print(preprocessed_text_sv_en_sv.take(10))
print(preprocessed_text_sv_en_sv.count())

[['resumption', 'of', 'the', 'session'], ['i', 'declare', 'resumed', 'the', 'session', 'of', 'the', 'european', 'parliament', 'adjourned', 'on', 'friday', '17', 'december', '1999,', 'and', 'i', 'would', 'like', 'once', 'again', 'to', 'wish', 'you', 'a', 'happy', 'new', 'year', 'in', 'the', 'hope', 'that', 'you', 'enjoyed', 'a', 'pleasant', 'festive', 'period.'], ['although,', 'as', 'you', 'will', 'have', 'seen,', 'the', 'dreaded', "'millennium", "bug'", 'failed', 'to', 'materialise,', 'still', 'the', 'people', 'in', 'a', 'number', 'of', 'countries', 'suffered', 'a', 'series', 'of', 'natural', 'disasters', 'that', 'truly', 'were', 'dreadful.'], ['you', 'have', 'requested', 'a', 'debate', 'on', 'this', 'subject', 'in', 'the', 'course', 'of', 'the', 'next', 'few', 'days,', 'during', 'this', 'part-session.'], ['in', 'the', 'meantime,', 'i', 'should', 'like', 'to', 'observe', 'a', "minute'", 's', 'silence,', 'as', 'a', 'number', 'of', 'members', 'have', 'requested,', 'on', 'behalf', 'of', '

In [13]:
import string #to remove punctuations
#reference: https://medium.com/@gulcanogundur/pyspark-word-count-b099106135a7 for punctuations and lambda use.

def lowercase_split_map(rdd):
    split_lines = rdd.flatMap(lambda line: line.lower().strip(string.punctuation).split(' '))
    mapped = split_lines.filter(lambda x: x.isalpha()) \
                .map(lambda words: (words.strip(string.punctuation), 1), split_lines) \
                .reduceByKey(lambda a, b: a + b) \
                .sortByKey()
    return mapped

word_count_sv_en_en = lowercase_split_map(lines_sv_en_en)
word_count_sv_en_en = word_count_sv_en_en.map(lambda x:(x[1],x[0]))

#print(word_count_sv_en_en.sortByKey(False).take(100))


word_count_sv_en_sv = lowercase_split_map(lines_sv_en_sv)
word_count_sv_en_sv = word_count_sv_en_sv.map(lambda x:(x[1],x[0]))

print(word_count_sv_en_sv.sortByKey(False).take(100))


[(1706328, 'att'), (1344865, 'och'), (1052440, 'i'), (942048, 'det'), (913312, 'som'), (912272, 'för'), (739577, 'av'), (696798, 'är'), (620508, 'en'), (540272, 'vi'), (514492, 'om'), (513971, 'de'), (510056, 'till'), (502907, 'jag'), (500877, 'den'), (482266, 'på'), (469207, 'har'), (426964, 'med'), (402917, 'inte'), (370117, 'ett'), (252522, 'detta'), (194851, 'kommer'), (193936, 'kan'), (181528, 'måste'), (148730, 'från'), (144093, 'europeiska'), (130340, 'vill'), (127621, 'så'), (125748, 'men'), (125226, 'också'), (124630, 'skulle'), (123821, 'när'), (123456, 'denna'), (119501, 'man'), (117352, 'herr'), (110630, 'kommissionen'), (110466, 'mycket'), (110462, 'alla'), (100930, 'sig'), (100210, 'dessa'), (93384, 'vara'), (93327, 'eller'), (89829, 'även'), (89100, 'andra'), (88969, 'gäller'), (80671, 'oss'), (80333, 'mot'), (80226, 'under'), (79907, 'därför'), (79281, 'här'), (79052, 'inom'), (77565, 'eu'), (75126, 'finns'), (74734, 'skall'), (73615, 'utan'), (69981, 'genom'), (67909, 

In [14]:
en_1 = lines_sv_en_en.zipWithIndex()
en_2 = en_1.map(lambda x:(x[1],x[0]))

sv_1 = lines_sv_en_sv.zipWithIndex()
sv_2 = sv_1.map(lambda x:(x[1],x[0]))

joined_en2_sv2 = en_2.join(sv_2)

print(joined_en2_sv2.take(10))



[(0, ('Resumption of the session', 'Återupptagande av sessionen')), (5, ("Please rise, then, for this minute' s silence.", 'Jag ber er resa er för en tyst minut.')), (10, ("Would it be appropriate for you, Madam President, to write a letter to the Sri Lankan President expressing Parliament's regret at his and the other violent deaths in Sri Lanka and urging her to do everything she possibly can to seek a peaceful reconciliation to a very difficult situation?", 'Skulle det vara möjligt för er, fru talman, att skriva ett brev till den srilankesiska presidenten i vilket parlamentets beklagande uttrycks över hans och de övriga brutala dödsfallen i Sri Lanka och uppmanar henne att göra allt som står i hennes makt för att få en fredlig lösning på en mycket komplicerad situation?')), (15, ('My question relates to something that will come up on Thursday and which I will then raise again.', 'Min fråga har att göra med något som kommer att behandlas på torsdag och som jag då kommer att ta upp ig

In [38]:
#joined_en2_sv2 = joined_en2_sv2.join(joined_en2_sv2).filter(lambda line_number,(en_ls,sv_rs): len(en_ls.split()) == len(sv_ls.split())

rdd57 = joined_en2_sv2.map(lambda x: (x[1][0], x[1][1]))


joined_en2_sv2 = joined_en2_sv2.filter(lambda x: len(x[1][0].split()) == len(x[1][1].split()));
joined_en2_sv2.__class__      

test = joined_en2_sv2.collect()

#print(joined_en2_sv2.toDF().take(10))

    

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 50.0 failed 4 times, most recent failure: Lost task 2.3 in stage 50.0 (TID 120, 192.168.2.145, executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 597, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 271, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/util.py", line 107, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-24-0afdaf69a105>", line 3, in <lambda>
AttributeError: 'tuple' object has no attribute 'split'

	at o